# Text Preprocessing for use with Topic Models

The text has already been cleaned.  This script will preprocess it - tokenize, remove stop words, add bigrams and trigrams, lemmatize.

In [17]:
import pandas as pd
import pickle
import nltk
import numpy as np

import TextCleaning
import LDAvariables
import stanza

import gensim


In [18]:
#I used this document to understand what was happening in pre-processing at a small scale. 
#Currently I deleted the original stanford model because I accidently override in wiht a small df

In [19]:
# load saved df.  df['working_abstract'] contains clean text.
df = pd.read_csv("../../data/original/working_federal_reporter_2020.csv")
df.reset_index(inplace = True)
df.rename(columns={'index':'original index'}, inplace=True)

/apps/software/standard/core/anaconda/2019.10-py3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1,7,13,14,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
wa='ABSTRACT'

In [21]:
df.dtypes
#df = df.astype(str)
df.dtypes

original index                  int64
PROJECT_ID                    float64
ABSTRACT                       object
FY.x                          float64
PROJECT_TERMS                  object
PROJECT_TITLE                  object
DEPARTMENT                     object
AGENCY                         object
IC_CENTER                      object
PROJECT_NUMBER                 object
PROJECT_START_DATE             object
PROJECT_END_DATE               object
CONTACT_PI_PROJECT_LEADER      object
OTHER_PIS                      object
CONGRESSIONAL_DISTRICT         object
DUNS_NUMBER                    object
ORGANIZATION_NAME              object
ORGANIZATION_CITY              object
ORGANIZATION_STATE             object
ORGANIZATION_ZIP               object
ORGANIZATION_COUNTRY           object
BUDGET_START_DATE              object
BUDGET_END_DATE                object
CFDA_CODE                      object
FY.y                          float64
FY_TOTAL_COST                 float64
FY_TOTAL_COS

In [22]:
#Create subset 
smalldf  = df[:10]
smalldf

,original index,PROJECT_ID,ABSTRACT,FY.x,PROJECT_TERMS,PROJECT_TITLE,DEPARTMENT,AGENCY,IC_CENTER,PROJECT_NUMBER,...,ORGANIZATION_CITY,ORGANIZATION_STATE,ORGANIZATION_ZIP,ORGANIZATION_COUNTRY,BUDGET_START_DATE,BUDGET_END_DATE,CFDA_CODE,FY.y,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS
0,0,89996.0,"This is a project to explore Game-based, Metap...",2008.0,Achievement; analog; base; Cognitive Science; ...,RUI: CYGAMES: CYBER-ENABLED TEACHING AND LEARN...,NSF,NSF,NaN,0814512,...,WHEELING,WV,26003-6243,UNITED STATES,NaN,NaN,47.076,2008.0,1999467.0,NaN
1,1,89997.0,Institution: Franklin Institute Science Museum...,2008.0,Active Learning; Child; Computer software; des...,ARIEL - AUGMENTED REALITY FOR INTERPRETIVE AND...,NSF,NSF,NaN,0741659,...,PHILADELPHIA,PA,19103-1115,UNITED STATES,NaN,NaN,47.076,2008.0,1799699.0,NaN
2,2,89998.0,Through programs (including small group conver...,2008.0,Address; Age; Birth; Brain; Caregivers; Child;...,BRIGHTER FUTURES: PUBLIC DELIBERATION ABOUT TH...,NSF,NSF,NaN,0813522,...,SAINT PAUL,MN,55102-1202,UNITED STATES,NaN,NaN,47.076,2008.0,1505858.0,NaN
3,3,89999.0,In partnership with the American Chemical Soci...,2008.0,Advanced Development; American; Chemicals; Che...,FOSTERING US-INTERNATIONAL COLLABORATIVE PARTN...,NSF,NSF,NaN,0838627,...,DURHAM,NC,27709-3757,UNITED STATES,NaN,NaN,47.049,2008.0,51000.0,NaN
4,4,90000.0,Amphibian populations around the world are exp...,2008.0,Amphibia; Central America; Communicable Diseas...,COLLABORATIVE RESEARCH: EVOLUTION OF AMPHIBIAN...,NSF,NSF,NaN,0815315,...,ITHACA,NY,14850-2820,UNITED STATES,NaN,NaN,47.074,2008.0,370996.0,NaN
5,5,90001.0,The Center for Molecular Interfacing (CMI) wil...,2008.0,Address; Architecture; Carbon Nanotubes; Catal...,CCI PHASE I: CENTER FOR MOLECULAR INTERFACING,NSF,NSF,NaN,0847926,...,ITHACA,NY,14850-2820,UNITED STATES,NaN,NaN,47.049,2008.0,1519821.0,NaN
6,6,90002.0,DRU: Integrated optimization of evacuation and...,2008.0,Accounting; Address; Affect; American; Area; b...,DRU: INTEGRATED OPTIMIZATION OF EVACUATION AND...,NSF,NSF,NaN,0826832,...,NEWARK,DE,19716-2553,UNITED STATES,NaN,NaN,47.075,2008.0,757499.0,NaN
7,7,90003.0,The Flora of China (FOC) is an international c...,2008.0,Accounting; Biodiversity; Botanicals; China; C...,FLORA OF CHINA,NSF,NSF,NaN,0813935,...,SAINT LOUIS,MO,63110-3420,UNITED STATES,NaN,NaN,47.074,2008.0,250000.0,NaN
8,8,90004.0,NaN,2008.0,Fellowship Program; Research,GRADUATE RESEARCH FELLOWSHIP PROGRAM,NSF,NSF,NaN,0852410,...,MINNEAPOLIS,MN,55455-2070,UNITED STATES,NaN,NaN,47.076,2008.0,4642186.0,NaN
9,9,90005.0,The overall goal of this project is to reconst...,2008.0,Alaska; Aleutian Islands; Arctic Regions; Area...,COLLABORATIVE RESEARCH: COUPLED GLACIAL AND LA...,NSF,NSF,NaN,0823522,...,FLAGSTAFF,AZ,86011-4130,UNITED STATES,NaN,NaN,47.050,2008.0,357187.0,NaN


In [23]:
import spacy

#need to install python -m spacy download en 

from spacy.lang.en import English

import en_core_web_sm
nlp = en_core_web_sm.load()


def getLemma(abstract):
    abstract = str(abstract)
    doc = nlp(abstract)
    listoflemma = []
    for word in doc:
        l = word.lemma_
        listoflemma.append(l)
    return listoflemma
smalldf['LEMMA_ABSTRACT'] = smalldf['ABSTRACT'].apply(getLemma)

smalldf['LEMMA_ABSTRACT']

smalldf.head()    


/apps/software/standard/core/anaconda/2019.10-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,original index,PROJECT_ID,ABSTRACT,FY.x,PROJECT_TERMS,PROJECT_TITLE,DEPARTMENT,AGENCY,IC_CENTER,PROJECT_NUMBER,...,ORGANIZATION_STATE,ORGANIZATION_ZIP,ORGANIZATION_COUNTRY,BUDGET_START_DATE,BUDGET_END_DATE,CFDA_CODE,FY.y,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS,LEMMA_ABSTRACT
0,0,89996.0,"This is a project to explore Game-based, Metap...",2008.0,Achievement; analog; base; Cognitive Science; ...,RUI: CYGAMES: CYBER-ENABLED TEACHING AND LEARN...,NSF,NSF,NaN,0814512,...,WV,26003-6243,UNITED STATES,NaN,NaN,47.076,2008.0,1999467.0,NaN,"[this, be, a, project, to, explore, Game, -, b..."
1,1,89997.0,Institution: Franklin Institute Science Museum...,2008.0,Active Learning; Child; Computer software; des...,ARIEL - AUGMENTED REALITY FOR INTERPRETIVE AND...,NSF,NSF,NaN,0741659,...,PA,19103-1115,UNITED STATES,NaN,NaN,47.076,2008.0,1799699.0,NaN,"[institution, :, Franklin, Institute, Science,..."
2,2,89998.0,Through programs (including small group conver...,2008.0,Address; Age; Birth; Brain; Caregivers; Child;...,BRIGHTER FUTURES: PUBLIC DELIBERATION ABOUT TH...,NSF,NSF,NaN,0813522,...,MN,55102-1202,UNITED STATES,NaN,NaN,47.076,2008.0,1505858.0,NaN,"[through, program, (, include, small, group, c..."
3,3,89999.0,In partnership with the American Chemical Soci...,2008.0,Advanced Development; American; Chemicals; Che...,FOSTERING US-INTERNATIONAL COLLABORATIVE PARTN...,NSF,NSF,NaN,0838627,...,NC,27709-3757,UNITED STATES,NaN,NaN,47.049,2008.0,51000.0,NaN,"[in, partnership, with, the, American, Chemica..."
4,4,90000.0,Amphibian populations around the world are exp...,2008.0,Amphibia; Central America; Communicable Diseas...,COLLABORATIVE RESEARCH: EVOLUTION OF AMPHIBIAN...,NSF,NSF,NaN,0815315,...,NY,14850-2820,UNITED STATES,NaN,NaN,47.074,2008.0,370996.0,NaN,"[amphibian, population, around, the, world, be..."


In [24]:
##############
#Helper Functions
#############################################

#This function was used previously, but we were unaware it had already been included in 'Cleaning', and is redundant.        
#def remove_institution(record):
#    """removes all instances of exact institution name from lowercase abstract string"""
#    org=record['ORGANIZATION_NAME']
#    if pd.notnull(org):
#        return record['working_abstract'].replace(org.lower(),'')
#    else:
#        return record['working_abstract']
    
def remove_custom_words(record,col_to_clean):
    """Designates stopwords for a particular abstract that contain embedded info e.g. author names and removes them from a lowercase abstract"""
    fields_to_replace=[]
    if type(record[col_to_clean])!=list:
        return np.nan
    else:
        #Main PI
        #Adds all words in the pis names, excluding initials (hence why the commas and periods must be replaced)
        if pd.notnull(record['CONTACT_PI_PROJECT_LEADER']):
            fields_to_replace.extend([x.lower() for x in record['CONTACT_PI_PROJECT_LEADER'].replace(',','').replace('.','').replace('-',' ').split() if len(x)>1])
        #Additional PIs
        #For each pi, which are split by semicolons, and format is last,first;  #Sometimes a middle initial
        if pd.notnull(record['OTHER_PIS']):
            for i in record['OTHER_PIS'].split(';'):
                i=i.strip() #Remove whitespace
                i=i.replace('.','')#Periods for initials
                i=i.replace(',','')#Commas between last, first
                i=i.replace('-',' ')#Remove hyphen in hypenated names to make separate words once tokens.
                fields_to_replace.extend([x.lower().strip() for x in i.split() if len(x)>1])
        return [x.lower() for x in record[col_to_clean] if not x.lower() in fields_to_replace]

def remove_first_x_tokens(tokened_abstract,bad_start_phrases,max_tokens_to_skip=3):
    """removes each bad_start_phrase occuring within max_tokens_to_skip of the front--phrases must be lowered.
    be careful calling this, as order matters! It always starts looking at the first token, which will change between runs.
    both tokened_abstract and each phrase in bad_start_phrases must be a list, not just a string
    eg the phrase 'overall project summary' and 'technical abstract' should be input as a list of lists: [ ['overall','project','summary'],['technical','abstract']] """
    assert [type(phrase)==list for phrase in bad_start_phrases] #Make sure not just a string
    assert [type(tokened_abstract)==list]
    if type(tokened_abstract)!=list:
        return np.nan
    else:
        for token_sequence in bad_start_phrases:
            #Look for a match within up to 3 tokens from the start. The reasoning here is some abstract start with numbers indicating sections
            #EG 8., 8.a, 8.1.1.--from EDA of first tokens
            for idx in range(0,max_tokens_to_skip):
                if tokened_abstract[idx:len(token_sequence)+idx]==token_sequence:
                    tokened_abstract=tokened_abstract[len(token_sequence)+idx:]
                    break
        return tokened_abstract

#Original list used to remove--must be updated now that lemmatization occurs before
#start_phrases_to_remove=[['section'],['abstract'],['contact','pd','pi'],['technical'],['nontechnical'],['non','technical'],
#                         ['project','summary','abstract'], ['overall','project','summary'],['project','abstract'],
#                        ['project','narrative'],['abstract'],['summary'],['description','provided','by','the','applicant'],
#                         ['description','provided','by','applicant'],['description','provided','by','candidate'],
#                         ['provided','by','investigator'], ['provided','by','the','investigator'],['description']]

start_phrases_to_remove=[['section'],['abstract'],['contact','pd','pi'],['nontechnical'],['non','technical'], ['non-technical'],['technical'],
                         ['project','summary','abstract'], ['overall','project','summary'],['project','abstract'],
                        ['project','narrative'],['abstract'],['summary'],['description','provide','applicant'],
                         ['description','provide','applicant'],['description','provide','candidate'],
                         ['provide','investigator'],['description']]

"""
#Prior function which used the spacy module--but spacy is not 'research-grade'
def lemmatize_spacy(doc,punctuation_or_token='token'):
"""
"""use spacy to lemmatize a document. token takes a list of strings and is then turned into a string once again. Punctuation takes one string with punctuation and parses by sentence"""
"""
    assert punctuation_or_token in ['token','punctuation']
    if punctuation_or_token=='token':
        sentence=sp(' '.join(doc))
    elif punctuation_or_token=='punctuation':
        sentence=sp(doc)
    new_tokens=[]
    for word in sentence:
        if word.pos_ in ['NOUN','VERB','ADJ','ADV']:
            new_tokens.append(word.lemma_)
        elif word.pos_ in ['PROPN','NUM','X','INTJ']:
            new_tokens.append(word.text)
    return new_tokens
"""    
def lemmatize_stanford(doc,pretokened=False,keep_numbers=True):
    """if pretokened, dont use this function, as it hasnt been adapted for it"""
    ##to compare lemmatization functions, try test cases 463, 40, and 2247 (iloc)
    assert not pretokened #If these are already tokened per another pipeline, this function won't work correctly
    new_tokens=[]
    if doc==' ': #Quirk that somehow two empty abstracts were not caught
        return np.nan #Produces null abstracts that can mess up your code if you're not careful
    else: 
        processed=nlp(doc)
        print(processed)
        for sent in processed.sentences:
            for word in sent.words:
                #If its a regular noun, verb, adj, or adverb, keep lemmatized form
                if word.pos in ['NOUN','VERB','ADJ','ADV']:
                    new_tokens.append(word.lemma)
                #If you decided to retain numbers, their lemma is kept here. Note that number catching isnt perfect by this lemmatizing.
                elif word.pos=='NUM' and keep_numbers:
                    new_tokens.append(word.lemma)
                #Exact phrases are kept here with no attempt at lemmatization: e.g. mars does not become mars, and hopefully scientific words e.g. chemicals will be tagged as propn, x, or intj if needed
                elif word.pos in ['PROPN','X','INTJ']: 
                    new_tokens.append(word.text)
                #Note that no other tokens are kept
        return new_tokens


def create_stopwords():
    """ creates list of stopwords. stop words include the general English list and any additional we see sneaking through
    #We no longer remove words specific to the corpus that do not aid in meaning like science/research"""
    
    stopWords = set(nltk.corpus.stopwords.words('english'))
    
    # format stop words the same way we formatted our corpus, ie. without apostrophes.  
    stop_wds = stopWords.copy()
    for word in stopWords:
        if "\'" in word:
            stop_wds.discard(word)
            stop_wds.add(word.replace("\'",""))
    
    # more stop words that do not add meaning to topics
    additional_stopwords=['another','well','addition', 'thus',
                      'specifically', 'similar','including',
                       'via','within', 'thus', 'particular', 'furthermore','include','also',
                      'includes','however','whether','due', 'may','overall', 'whether','could',
                      'many','finally', 'several', 'specific', 'additional', 'therefore', 'either', 'various',
                       'within', 'among', 'would','research','project','subproject','1','cells','cell','2','data',
                        'studies','development','study','specific','health','cancer','disease','3','provide','clinical','aim','based',
                        'provided', 'program','human','pateints','understanding'] 
        
    sw = stop_wds.union(additional_stopwords)
    
    return sw

def remove_stopwords(doc, stop_words):
    """remove stopwords"""
    #If no acceptable tokens, this is np.nan
    if type(doc)!=list:
        return np.nan
    
    return [word for word in doc if word not in stop_words] 

def apply_n_grams(abstract,function):
    """apply an n-gram--could be bi or tri gram--to abstract"""
    #Non lists--aka no acceptable tokens
    if type(abstract)==list:
        return function[abstract]
    else:
        return abstract


    import re

def clean_up_tokens(doc):
    """determines for each doc which tokens to clean up formatting further in keep_token, and decides which of these cleaned up tokens will be kept"""
    kept_tokens=[]
    #Ignores documents that are abstracts with no valid tokens
    if type(doc)!=list:
        return np.nan
    else:
        for token in doc:
            keep,altered_token=keep_token(token)
            if keep:
                kept_tokens.append(altered_token)
        return kept_tokens
        
def keep_token(token):
    """strips hyphens, replaces internal hyphens with _, turns non-alphanumeric tokens into alphanumerics', strips leading _ if produced by alphamumeric,
    then removes those updated tokens that: are numeric but not length 4, are tokens related to college names (see below) or are less than length 2."""
    token=token.strip('- ') #Removes leading and trailing hyphens
    token=token.replace('-','_')
    if not str.isalnum(token):
        token=re.sub(r'\W+', '', token)
    token=token.strip('_')
    #Names of universities
    if 'university' in token or 'college' in token or 'universities' in token:
        return (not token in college_tokens, token)
    if str.isnumeric(token):
        #keep years
        return (len(token)==4,token)
    else:
        #Keep anything that is alphanumeric or alpha if its over length 2--allows mixed types e.g. h1n1
        return (len(token)>=2,token)



#Any specific university word is removed--schools within college/university, college names, etc. that cannot apply to multiple schools
#This list was generated from all tokens that contained the string 'college','university',or 'universities'. The commented out parts of the list are terms
#That could be considered "generic" ie apply to more than one school
college_tokens=[#'college',
'aga_khan_university',
'ahmadu_bello_university',
'alabama_aamp_university',
'albert_einstein_college',
'alcorn_state_university',
'american_college_obstetricians',
'american_college_surgeons',
'americancollege',
#'amongcollege',
#'anduniversity',
'anne_molloy_trinity_college',
#'atuniversity',
'auburn_university',
'auburn_university_alabama',
'auburn_university_au',
'auburn_university_auburn',
'auburn_university_montgomery',
'auburn_university_tuskegee_university',
'augustana_college',
#'auniversity',
'babes_bolyai_university',
'barnard_college',
'baruch_college',
'bates_college',
'baylor_college',
'baylor_college_dentistry',
'baylor_college_medicine',
'baylor_college_ofmedicine',
'baylor_collegeof',
'baylorcollege_medicine',
'baylorcollege_medicine_bcm',
'ben_gurion_university',
'benedict_college',
'benedict_college_historically_black',
'berea_college',
'binghamton_university',
#'black_colleges',
'board_trinity_college',
'bostonuniversity',
'bowdoin_college',
'brownuniversity',
'bryn_mawr_college',
'bucknell_university',
#'cape_universities',
'cardiff_university',
'carleton_college',
'carnegie_mellon_university',
'carver_college',
'carver_college_medicine',
'case_western_reserveuniversity',
'case_westernreserve_university',
'catholic_university',
'cerritos_college',
'charles_drew_university',
'chulalongkorn_university',
'chulalongkorn_university_bangkok_thailand',
'claflin_university',
'claremont_colleges',
'clark_atlanta_university',
'colby_college',
'colby_sawyer_college',
#'college',
#'college',
#'college_american_pathologists',
#'college_arts',
#'college_arts_sciences',
'college_brockport',
'college_dentistry_nyucd',
'college_dentistry_ufcd',
#'college_goer',
#'college_graduates',
#'college_letters',
#'college_letters_arts_sciences',
#'college_letters_sciences',
'college_lewiston',
#'college_liberal_arts',
'college_london',
'college_medicine_aecom',
'college_medicine_uccom',
'college_menominee_nation',
#'college_optometry',
#'college_osteopathic_medicine',
'college_park_umcp',
'college_park_umd',
#'college_physicians_surgeons',
#'college_rheumatology_acr',
'college_south_hadley',
#'college_sports_medicine',
'college_st_scholastica',
'college_staten_island',
#'college_students_basics',
#'college_veterinary_medicine',
#'college_veterinary_pathologists',
#'college_veterinarymedicine',
'college_wcmc',
'college_wcmc_rockefeller_university',
'college_william_mary',
'college_wisconsin_mcw',
'college_wooster',
#'collegeand',
#'collegeof',
#'collegeof_medicine',
#'colleges',
#'colleges_arts_sciences',
#'colleges_chicago',
#'colleges_dentistry',
#'colleges_dentistry_medicine',
#'colleges_optometry',
'colleges_rcc_umb', #Iffy--not sure what this is
#'colleges_schools',
#'colleges_universities',
#'colleges_universities_hacu',
#'collegesand',
#'collegestudent',
'columbia_university',
'columbiauniversity',
'comanche_nation_college',
#'communitycollege',
'creighton_university',
'cross_university',
'cross__university',
'cuny_hunter_college',
'del_mar_college',
'depaul_university',
'dine_college',
'din_college',
'diné_college',
'diplomate_american_college',
'doane_college',
'doron_levy_university_maryland',
'dukeuniversity',
'eckerd_college',
'emoryuniversity',
'famu_fsu_college',
'fort_lewis_college',
'franklin_marshall_college',
'fudan_university',
'fudan_university_shanghai',
'fudan_university_shanghai_china',
'gallaudet_university',
'george_mason_university',
'george_washington_university',
'georgetown_howard_universities',
'georgetown_university',
'georgia_regents_university',
'gettysburg_college',
#'grant_universities_aplu',
'gu_howard_university',
'hackensack_university',
'hampton_university',
'hanyang_university',
'hartnell_college',
'harvarduniversity',
'harvey_mudd_college',
#'historically_black_college',
#'historically_black_colleges',
#'historically_black_colleges_universities',
'hokkaido_university',
'hold_bates_college',
'hold_colby_sawyer_college',
'hold_stonehill_college_easton',
'honors_college',
'houston_baylor_college',
'hunter_college',
'imperial_college',
'imperial_college_london',
'imperial_college_london_uk',
#'incollege',
'indiana_university',
'indianauniversity',
#'inspect_certified_college',
#'inter_college',
#'inter_university',
#'inter_university_consortium_political',
#'interuniversity',
#'interuniversity_consortium_political',
#'intra_university',
'james_cook_university',
'james_madison_university',
'jeffersonuniversity',
'john_jay_college',
'johns_hopkinsuniversity',
'kennesaw_state_university',
'king_college_london',
'kwame_nkrumah_university',
'kyoto_university',
'kyushu_university',
'langston_university',
'lehman_college',
'lehman_college_city',
'lehman_college_cuny',
'lemoyne_owen_college',
'lewis_clark_college',
#'liberal_art_college',
'louisiana_universities_marine',
'loyola_marymount_university',
'loyola_university',
'loyola_university_chicago',
'macalester_college',
'makerere_university',
'makerere_university_kampala_uganda',
'makerere_university_uganda',
'makerereuniversity',
'marquette_university',
'marquette_university_milwaukee',
'mbarara_university',
'mcgill_university',
'mcmaster_university',
'medgar_evers_college',
'medical_colleges_aamc',
'medicalcollege',
'medicaluniversity_south_carolina',
'medicine_yeshiva_university',
'meharrymedical_college',
'mellon_university',
'mellonuniversity',
'mexico_highlands_university',
'miami_dade_college',
'middlebury_college',
'millsaps_college',
'monash_university',
'monash_university_australia',
#'montana_tribal_college',
#'montana_tribal_colleges',
'montclair_state_university',
'morehouse_college',
'morehouse_college_spelman_college',
'mount_holyoke_college',
'msm_tuskegee_university',
'mt_marty_college',
'muhimbili_university',
#'multi_university',
#'muniversity',
'nakoda_college',
'nanyang_technological_university',
'nazarene_university',
'nazareth_college',
#'non_college',
#'non_university',
'northern_arizona_university',
'northern_kentucky_university',
'northshore_university',
'northshore_university_healthsystem',
'northwest_nazarene_university',
'northwestern_university',
'norwich_university',
#'ofuniversity',
'oglala_lakota_college',
'ohio_stateuniversity',
'old_dominion_university',
'olin_college',
#'otheruniversity',
#'participatinguniversity',
'pasadena_city_college',
'peking_university',
'peking_university_beijing_china',
'pennsylvania_college_optometry',
#'phduniversity',
#'polytechnic_university',
#'post__college',
'prairie_view_university',
#'pre_college',
#'pre_university',
#'pre__college',
#'precollege',
'queens_college',
'regents_university',
'researchuniversity',
'rockefeller_university',
'rockefeller_university_memorial_sloan',
'rockefeller_university_ru',
'rockefeller_university_weill_cornell',
'rockefelleruniversity',
'royal_college_surgeons',
'rutgers_university',
'rutgersuniversity',
'saddleback_college',
'saginaw_chippewa_tribal_college',
'saint_michael_college',
'salish_kootenai_college',
'salve_regina_university',
'sawyer_college',
#'scienceuniversity', #This is likely ohsu, as bellow, but for parsimony, this is kept
'scienceuniversity_ohsu',
'serc_carleton_college',
'shams_university',
'shams_university_cairo_egypt',
'shanghai_jiaotong_university',
'simon_fraser_university',
'sinte_gleska_university',
'sisseton_wahpeton_college',
'sitting_bull_college',
'skc_tribal_college',
'sokoine_university',
'south_africa_university_witwatersrand',
'southern_illinois_university_carbondale',
'southern_illinois_university_edwardsville',
'southern_methodist_university',
'spelman_college',
'st_edward_university',
'st_mary_college',
'st_olaf_college',
'st_philip_college',
'stanforduniversity',
'state_university_dominguez', #Specific university
#'stateuniversity', #This could be any state
'stellenbosch_university',
'stellenbosch_university_south_africa',
'stonehill_college',
'stonehill_college_easton_massachusetts',
'stony_brook_university',
'swarthmore_college',
'tarrant_county_college',
'tel_aviv_university',
'templeuniversity',
'texas_a_university',
'texas_southmost_college',
'texas_university_kingsville',
#'thecollege',
#'theuniversity',
'theuniversity_california_san',
'theuniversity_colorado',
'theuniversity_maryland',
'theuniversity_michigan',
'theuniversity_minnesota',
'theuniversity_north_carolina',
'theuniversity_pennsylvania',
'theuniversity_pittsburgh',
'tougaloo_college',
#'touniversity',
#'triangle_universities_nuclear', #this is a government research center
#'tribal_college',
'tribal_college_haskell_indian', #specific university
#'tribal_colleges',
#'tribal_colleges_universities',
#'tribal_colleges_universities_tcus',
'trinity_college',
'trinity_college_arts_sciences',
'trinity_college_dublin',
'tsinghua_university',
'tsinghua_university_beijing',
'tsinghua_university_beijing_china',
'tsinghua_university_china',
#'tsinghua_university_prof_roberto',
'tulaneuniversity',
'tuskegee_universities',
'tuskegee_university',
'tuskegee_university_hbcu',
'uams_colleges',
'ucsf_makerere_university',
'umbc_university_maryland',
'uniformed_services_university',
'united_negro_college',
#'universities',
#'universities_aau', #this is an association of universities, not a university
#'universities_hbcu',
'universities_kansas_ku',
#'universitiesand',
#'universitiesin',
#'university',
#'university',
'university_alabama_birmingham',
'university_alabama_huntsville',
'university_alabama_tuscaloosa',
'university_alabama_ua',
'university_alaska_anchorage',
'university_alaska_fairbanks',
'university_albany_suny',
'university_arizona_ua',
'university_arkansas_fayetteville',
'university_arkansas_pine',
'university_arkansas_ua',
'university_buffalo_suny',
'university_buffalo_ub',
'university_california_berkeley',
'university_california_davis',
'university_california_irvine',
'university_california_los',
'university_california_merced',
'university_california_riverside',
'university_california_san',
'university_california_sanfrancisco',
'university_california_santa',
'university_cincinnati_cincinnati',
'university_college_dublin',
'university_college_london',
'university_colorado_anschutz',
'university_colorado_boulder',
'university_colorado_denver',
'university_connecticut_uconn',
'university_feinberg_school',
'university_florida_gainesville',
'university_florida_uf',
'university_fullerton_csuf',
'university_georgia_athens',
'university_georgia_uga',
'university_hawaii_hilo',
'university_hawaii_manoa',
'university_hawaii_uh',
'university_hospitals_cleveland',
'university_houston_downtown',
'university_houston_uh',
'university_illinois_chicago',
'university_illinois_urbana',
'university_indianapolis_iupui',
'university_kansas_ku',
'university_kansas_lawrence',
'university_kingsville',
'university_langone_medical',
'university_louisiana_lafayette',
'university_louisiana_monroe',
'university_maryland',
'university_maryland_baltimore',
'university_maryland_baltimore_county',
'university_maryland_baltimore_umb',
'university_maryland_eastern_shore',
'university_maryland_greenebaum',
'university_maryland_marlene_stewart', 
'university_maryland_umd',
'university_massachusetts_amherst',
'university_massachusetts_dartmouth',
'university_massachusetts_lowell',
'university_massachusetts_umass',
'university_miami_miller',
'university_miami_um',
'university_michigan_ann',
'university_michigan_dearborn',
'university_michigan_um',
'university_minnesota',
'university_minnesota_duluth',
'university_minnesota_masonic',
'university_minnesota_minneapolis',
'university_minnesota_twin',
'university_minnesota_umn',
'university_missouri__columbia',
'university_missouri_columbia',
'university_missouri_kansas',
'university_missouri_mu',
'university_missouri_rolla',
'university_missouri_st',
'university_nebraska_lincoln',
'university_nebraska_omaha',
'university_nevada_las',
'university_nevada_reno',
'university_northcarolina_chapel',
'university_northridge_csun',
'university_ofalabama',
'university_ofcalifornia',
'university_ofcolorado',
'university_ofmichigan',
'university_ofminnesota',
'university_ofpennsylvania',
'university_ofrochester',
'university_oftexas',
'university_ofwashington',
'university_ofwashington_uw',
'university_ofwisconsin',
'university_ofwisconsin_madison',
'university_oklahoma_norman',
'university_oklahoma_ou',
'university_pennsylvania_upenn',
'university_pittsburgh_pitt',
'university_singapore_nus',
'university_singapore_singapore',
'university_tennessee_chattanooga',
'university_tennessee_knoxville',
'university_tennessee_memphis',
'university_texas_arlington',
'university_texas_austin',
'university_texas_brownsville',
'university_texas_dallas',
'university_texas_el',
'university_texas_pan',
'university_texas_rio',
'university_texas_southwestern',
'university_texas_tyler',
'university_toronto_toronto',
'university_venda',
'university_vermont_burlington',
'university_vermont_uvm',
'university_virginia_charlottesville',
'university_virginia_uva',
'university_washington_seattle',
'university_washington_uw',
'university_waterloo',
'university_west_indies',
'university_wisconsin_carbone',
'university_wisconsin_eau',
'university_wisconsin_madison',
'university_wisconsin_milwaukee',
'university_wisconsin_oshkosh',
'university_wisconsin_platteville',
'university_wisconsin_stout',
'university_witwatersrand',
'university_witwatersrand_south_africa',
'university_witwatersrand_wits',
#'universityabstract',
#'universityand',
#'universitycareer',
#'universityco',
#'universityhospitals',
#'universityin',
#'universityintellectual',
#'universitymedical',
#'universityof',
'universityof_california_san',
'universityof_chicago',
'universityof_colorado',
'universityof_kentucky',
'universityof_michigan',
'universityof_minnesota',
'universityof_pennsylvania',
'universityof_pittsburgh',
'universityof_washington',
#'universityproposal',
#'universityresources',
#'universitys',
#'universityschool_medicine',
#'universitytitle',
'urmc_college_arts',
'vanderbiltuniversity',
'virginia_commonwealth_university',
'wake_forest_university',
'washingtonuniversity',
'wayne_stateuniversity',
'weinberg_college_arts',
'wellesley_college',
'wesley_college',
'western_ontario_mcmaster_universities',
'western_ontario_mcmasters_universities',
'westminster_college',
#'withuniversity',
'xiamen_university',
'xiamen_university_china',
'yaleuniversity',
'yeshiva_university',
'yonsei_university',
'yonsei_university_seoul_south',
'yorkuniversity']

In [25]:
smalldf['LEMMA_ABSTRACT'].fillna(value=np.nan,inplace=True)
no_pis=smalldf.apply(lambda x: remove_custom_words(x,'LEMMA_ABSTRACT'),axis=1)

#no_pis.head()

In [26]:
no_pis_no_start=no_pis.apply(remove_first_x_tokens,args=[start_phrases_to_remove])

In [27]:
#Remove stopwords--nltk and those added on in 'additional_stopwords' function
stopWords = create_stopwords()
#tokened_docs_nostop = TextCleaning.remove_stopwords(no_pis, stopWords) #Old code--does not account for non list documents
tokened_docs_nostop = no_pis_no_start.apply(remove_stopwords,args=[stopWords])
smalldf['tokened_docs_nostop'] = tokened_docs_nostop

#smalldf['tokened_docs_nostop']

/apps/software/standard/core/anaconda/2019.10-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [28]:
#Note that this is not trained on null abstracts
bigram = gensim.models.Phrases(smalldf['tokened_docs_nostop'].dropna(), min_count=5, threshold=100) # higher threshold fewer phrases.
#This function will return a bigram if
bigram_docs=smalldf['tokened_docs_nostop'].apply(lambda x: apply_n_grams(x,bigram))
trigram = gensim.models.Phrases(bigram_docs.dropna(), threshold=100)  
tri_docs =bigram_docs.apply(lambda x: apply_n_grams(x,trigram))
smalldf['tns_bi_tri_docs'] = tri_docs
##################
#Create bigrams and trigrams
###################
#Note that this is not trained on null abstracts
bigram = gensim.models.Phrases(smalldf['tokened_docs_nostop'].dropna(), min_count=5, threshold=100) # higher threshold fewer phrases.
#This function will return a bigram if
bigram_docs=smalldf['tokened_docs_nostop'].apply(lambda x: apply_n_grams(x,bigram))
trigram = gensim.models.Phrases(bigram_docs.dropna(), threshold=100)  
tri_docs =bigram_docs.apply(lambda x: apply_n_grams(x,trigram))
smalldf['tns_bi_tri_docs'] = tri_docs
smalldf['tns_bi_tri_docs'] 

/apps/software/standard/core/anaconda/2019.10-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/apps/software/standard/core/anaconda/2019.10-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


0    [explore, game, -, base, ,, metaphor, enhanced...
1    [institution, :, franklin, institute, science,...
2    [(, small, group, conversation, ,, citizen, co...
3    [partnership, american, chemical, society, (, ...
4    [amphibian, population, around, world, experie...
5    [center, molecular, interfacing, (, cmi, ), en...
6    [dru, :, integrate, optimization, evacuation, ...
7    [flora, china, (, foc, ), international, colla...
8                                                [nan]
9    [goal, reconstruct, low, -, frequency, behavio...
Name: tns_bi_tri_docs, dtype: object

In [29]:
import re
smalldf['clean_tokens']=smalldf['tns_bi_tri_docs'].apply(clean_up_tokens)

/apps/software/standard/core/anaconda/2019.10-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [30]:
# save processed text
smalldf.to_pickle("spacy_processed_smalldataset_stanford_lemma.pkl")

In [31]:
smalldf.head()

,original index,PROJECT_ID,ABSTRACT,FY.x,PROJECT_TERMS,PROJECT_TITLE,DEPARTMENT,AGENCY,IC_CENTER,PROJECT_NUMBER,...,BUDGET_START_DATE,BUDGET_END_DATE,CFDA_CODE,FY.y,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS,LEMMA_ABSTRACT,tokened_docs_nostop,tns_bi_tri_docs,clean_tokens
0,0,89996.0,"This is a project to explore Game-based, Metap...",2008.0,Achievement; analog; base; Cognitive Science; ...,RUI: CYGAMES: CYBER-ENABLED TEACHING AND LEARN...,NSF,NSF,NaN,0814512,...,NaN,NaN,47.076,2008.0,1999467.0,NaN,"[this, be, a, project, to, explore, Game, -, b...","[explore, game, -, base, ,, metaphor, enhanced...","[explore, game, -, base, ,, metaphor, enhanced...","[explore, game, base, metaphor, enhanced, game..."
1,1,89997.0,Institution: Franklin Institute Science Museum...,2008.0,Active Learning; Child; Computer software; des...,ARIEL - AUGMENTED REALITY FOR INTERPRETIVE AND...,NSF,NSF,NaN,0741659,...,NaN,NaN,47.076,2008.0,1799699.0,NaN,"[institution, :, Franklin, Institute, Science,...","[institution, :, franklin, institute, science,...","[institution, :, franklin, institute, science,...","[institution, franklin, institute, science, mu..."
2,2,89998.0,Through programs (including small group conver...,2008.0,Address; Age; Birth; Brain; Caregivers; Child;...,BRIGHTER FUTURES: PUBLIC DELIBERATION ABOUT TH...,NSF,NSF,NaN,0813522,...,NaN,NaN,47.076,2008.0,1505858.0,NaN,"[through, program, (, include, small, group, c...","[(, small, group, conversation, ,, citizen, co...","[(, small, group, conversation, ,, citizen, co...","[small, group, conversation, citizen, conferen..."
3,3,89999.0,In partnership with the American Chemical Soci...,2008.0,Advanced Development; American; Chemicals; Che...,FOSTERING US-INTERNATIONAL COLLABORATIVE PARTN...,NSF,NSF,NaN,0838627,...,NaN,NaN,47.049,2008.0,51000.0,NaN,"[in, partnership, with, the, American, Chemica...","[partnership, american, chemical, society, (, ...","[partnership, american, chemical, society, (, ...","[partnership, american, chemical, society, acs..."
4,4,90000.0,Amphibian populations around the world are exp...,2008.0,Amphibia; Central America; Communicable Diseas...,COLLABORATIVE RESEARCH: EVOLUTION OF AMPHIBIAN...,NSF,NSF,NaN,0815315,...,NaN,NaN,47.074,2008.0,370996.0,NaN,"[amphibian, population, around, the, world, be...","[amphibian, population, around, world, experie...","[amphibian, population, around, world, experie...","[amphibian, population, around, world, experie..."


In [32]:
############
#Create datasets for analysis
############

#All data
docs = smalldf['clean_tokens'].dropna() #<--If you're having issues merging with a prior dataset, note that this will NOT be the same length as the df overall, because of nulls, so be careful appending
id2word, corpus = LDAvariables.createLDAvars(docs)
pickle.dump([corpus, id2word, docs], open('lda_data_stanford_lemma.sav','wb'))

#The two IDs for abstracts that were a space
print('Abstract IDs with no lemmas in them')
print(set(range(len(docs)))-set(docs.index))

#Just nsf
nsf_docs=smalldf.groupby('AGENCY').get_group('NSF')['final_tokens'].dropna()
id2word, corpus = LDAvariables.createLDAvars(nsf_docs)
pickle.dump([corpus, id2word, nsf_docs], open('nsf_stanford_lemma.sav','wb'))

Abstract IDs with no lemmas in them
set()


KeyError: 'final_tokens'